In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib

# Import Dataset

In [5]:
df = pd.read_csv('wedding_souvenirs_improved.csv')

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [11]:
print(X[0])

['Wedding Candle' 'Wax' 588 17.03867428120558]


In [12]:
products = {}

for item in X:
    product_name, raw_material, _, _ = item  # Unpack the values from the list
    if product_name not in products:
        products[product_name] = raw_material

print(products)

{'Wedding Candle': 'Wax', 'Custom Mug': 'Ceramic', 'Photo Frame': 'Wood', 'Custom Keychain': 'Engraving', 'Custom Coaster': 'Sealant'}


# Encode independent variable

In [3]:
categorical_columns = [0, 1]

# Column transformer to handle encoding and scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('num', StandardScaler(), slice(len(categorical_columns), X.shape[1]))
    ]
)

# Create Pipeline

In [4]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Train the model

In [7]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

joblib.dump(pipeline, 'pipeline.pkl')

['pipeline.pkl']

# Check Accuracy

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print the evaluation results
print(f"Mean Absolute Error (MAE): %.2f" %mae)
print(f"Mean Squared Error (MSE): %.2f" %mse)
print(f"Root Mean Squared Error (RMSE): %.2f" % rmse)
print("R² Score: " + str(r2))


Mean Absolute Error (MAE): 0.25
Mean Squared Error (MSE): 0.09
Root Mean Squared Error (RMSE): 0.30
R² Score: 0.9999127248573584


# Predict A Product

In [4]:
import sys
import json
import pandas as pd
from joblib import load

try:
    pipeline = load('pipeline.pkl')
    input_data = json.loads(sys.argv[1])

    custom_data = pd.DataFrame({
        'Product Name': [input_data['product_name']],
        'Raw Material Name': [input_data['raw_material_name']],
        'Raw Material Amount': [input_data['raw_material_amount']],
        'Labor Hours': [input_data['labor_hours']],
    })

    prediction = pipeline.predict(custom_data)
    
    result = {'predicted_products_made': prediction[0]}
    print(json.dumps(result))

except Exception as e:
    print(json.dumps({'error': str(e)}), file=sys.stderr)
    sys.exit(1)

ImportError: cannot import name 'joblib' from 'sklearn.externals' (c:\Python311\Lib\site-packages\sklearn\externals\__init__.py)